# 3.2 RecyclerView - Selection

## **Implementacja `RecyclerView`**

Naszą aplikację rozpoczniemy od utworzenia layoutu `main_activity.xml`

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    tools:context=".MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content" />

</LinearLayout>
```

Następnie dodajmy plik `rv_item.xml`, który będzie zawierał zdefiniowany layout pojedynczego elementu `RecyclerView`

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:orientation="horizontal"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/numberText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:text="default text"
        android:textSize="24sp"
        android:layout_gravity="center"
        android:textAlignment="center"
        android:layout_margin="16dp"
        android:background="@drawable/item_background"
        android:textStyle="bold"/>

</LinearLayout>
```

Będziemy posiadali tylko jedno pole `TextView`, w który wyświetlimy liczbę. Przejdźmy do `MainActivitry` i dodajmy funkcję zwracającą listę elementów, które będziemy wyświetlać.

```java
    private LinkedList<Integer> createList(){
        LinkedList<Integer> numbers = new LinkedList<>();
        for(int i = 0; i < 50; i++)
            numbers.add(i);
        return  numbers;
    }
```

Następnie zaimplementujmy klasę `NumberListAdapter`

```java
package pl.edu.uwr.pum.recyclerviewselectorjava;

import android.view.LayoutInflater;
import android.view.View;
import android.view.ViewGroup;
import android.widget.TextView;

import androidx.annotation.NonNull;
import androidx.recyclerview.selection.ItemDetailsLookup;
import androidx.recyclerview.selection.SelectionTracker;
import androidx.recyclerview.widget.RecyclerView;

import java.util.LinkedList;

public class NumberListAdapter extends RecyclerView.Adapter<NumberListAdapter.NumberListViewHolder>{

    private final LinkedList<Integer> numberList;

    public NumberListAdapter(LinkedList<Integer> numberList){
        this.numberList = numberList;
    }

    @NonNull
    @Override
    public NumberListAdapter.NumberListViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new NumberListViewHolder(LayoutInflater
                .from(parent.getContext())
                .inflate(R.layout.rv_item, parent, false));
    }

    @Override
    public void onBindViewHolder(@NonNull NumberListAdapter.NumberListViewHolder holder, int position) {
        holder.bind(numberList.get(position));
    }

    @Override
    public int getItemCount() {
        return numberList.size();
    }

    protected static class NumberListViewHolder extends RecyclerView.ViewHolder {

        private final TextView textView;

        public NumberListViewHolder(@NonNull View itemView) {
            super(itemView);
            textView = itemView.findViewById(R.id.numberText);
        }

        public void bind(int value){
            textView.setText(String.valueOf(value));
        }
    }
}

```

Tym razem nasz `Adapter` nie przyjmuje `Context` w konstruktorze. Do instancji `Context`, w metodzie `onCreateViewHolder` odwołujemy się przez `parent.getContext()`. W klasie `ViewHolder` mamy funkcję `bind` łączącą dane z odpowiednimi polami layoutu, jest ona następnie wywołana w metodzie `onBindViewHolder`.

## **Implementacja `Selector`**

W pierwszym kroku przejdźmy do pliku `build.gradle` i w bloku `dependencies` dodajmy wpis

```kotlin
    implementation 'androidx.recyclerview:recyclerview-selection:1.1.0'
```

Po zmodyfikowaniu skryptu `gradle` musimy wykonać synchronizację (**Sync Project with Gradle Files**). Chcemy uzyskać możliwość zaznaczania elementów i zmieniania ich właściwości. W pierwszym kroku musimy zdecydować się na typ klucza który będziemy wykorzystywać:
- `String` - wykorzystywany gdy mamy identyfikator `String`
- `Long` - wykorzystywamy gdy mamy identyfikator numeryczny
- `Parcelable` - gdy wykorzystujemy jako klucz np. `Uri`
W naszym przypadku wykorzystamy `Long` i właściwość `position` naszego `RecyclerView`.

Przechodzimy do klasy `NumberListAdapter`, w pierwszym kroku musimy ustawić `hasStableIds` - zmiana na `true` oznacza że każdy element może być reprezentowany przez unikalny klucz. Do konstruktora dodajemy

```java
    public NumberListAdapter(LinkedList<Integer> numberList){
        this.numberList = numberList;
        setHasStableIds(true);
    }
```

Następnie chcemy wykorzystać właściwość `position` jako identyfikator, w tym celu nadpisujemy metodę `getItemId`

```java
    @Override
    public long getItemId(int position) {
        return position;
    }
```

Kolejnym krokiem jest implementacja klasy `KeyProvider`, tutaj skorzystamy z domyślnej implementacji `StableIdKeyProvider`. Następnie musimy dostarczyć implementację `ItemDetailsLookup` - klasa ta zawiera informacje o zaznaczonych przez użytkownika elementach. Dodajmy nowy plik do projektu - `NumberItemDetailsLookup.java`.

Klasa `NumberItemDetailsLookup` musi rozszerzać klasę `ItemDetailsLookup`

```java
public class NumberItemDetailsLookup extends ItemDetailsLookup<Long> {}
```

W klasie będziemy potrzebować pola `RecyclerView` reprezentujący naszą listę

```java
    private final RecyclerView recyclerView;

    public NumberItemDetailsLookup(RecyclerView recyclerView) {
        this.recyclerView = recyclerView;
    }
```

Samo zaznaczanie w tej bibliotece bazuje na `MotionEvent` który musimy zmapować na nasz `ViewHolder`. W tym celu musimy naspisać metodę `getItemDetails`

```java
    public ItemDetails<Long> getItemDetails(@NonNull MotionEvent e) {}
```

W pierwszym kroku potrzebujemy dostać się do naszego `ViewHolder` - zapiszemy go w obiekcie `View`. Możemy się do niego dostać wywołując metodę `findChildViewHolder` z klasy `RecyclerView`

```java
        View view = recyclerView.findChildViewUnder(e.getX(), e.getY());
```

Metoda ta przyjmuje dwa argumenty
- `x` pozycję horyzontalną wyrażoną w pikselach
- `y` pozycję wertykalną wyrażoną w pikselach

Przekazując instancję `MotionEvent` z wywołanymi metodami `getX` i `getY` możemy ustalić który element został zaznaczony. Następnie sprawdzamy czy udało się zwrócić `ViewHolder` w warunku `if`, jeżeli wszystko poszło pomyślnie metoda zwraca obiekt typu `ItemDetails`, w przeciwnym razie zwracamy `null`

```java
        return ((NumberListAdapter.NumberListViewHolder)recyclerView.getChildViewHolder(view)).getItemDetails();
```

Obiekt `ItemDetails` musi zawierać dwie informacje:
- `position` - pozycję elementu `RecyclerView`
- `selectionId` - unikalny identyfikator elementu `RecyclerView`

Aby otrzymać te elementy do klasy `NumberListAdapter.NumberViewHolder` dodajemy metodę `getItemDetails` zwracającą `ItemDetail`.

```java
        public ItemDetailsLookup.ItemDetails<Long> getItemDetails() {
            return new ItemDetailsLookup.ItemDetails<Long>() {
                @Override
                public int getPosition() {
                    return getAdapterPosition();
                }

                @NonNull
                @Override
                public Long getSelectionKey() {
                    return getItemId();
                }
            };
        }
```

Zwracany obiekt (klasa anonimowa) musi implementować dwie metody. Aktualną pozycję wyciągamy przez wyołanie metody `getAdapterPosition` - metoda zwraca `Adapter` który jako ostatni był powiązany z tym `ViewHolder` lub `null` jeżeli takiego powiązanie nie było. W metodzie `getSelectionKey` zwracamy wywołanie metody `getItemId` klasy `RecyclerView`.

Kolejnym krokiem będzie zmienienie koloru tła zaznaczonego elementu. Możemy to zdefiniować w pliku `xml`. Do folderu **res -> drawable** dodajemy nowy plik **Drawable Resource File** (`item_background.xml`), jako **root element** wybieramy `selector`. Chcemy zdefiniować dwa kolory i ustawić jeden z nich dla stanu aktywnego

```xml
<?xml version="1.0" encoding="utf-8"?>
<selector xmlns:android="http://schemas.android.com/apk/res/android">
    <item android:drawable="@android:color/holo_blue_bright" android:state_activated="true" />
    <item android:drawable="@android:color/white" />
</selector>
```

Kolor stanu aktywnego ustawiamy przez zmianę domyślnej wartości `android:state_activated` na `true`. W ten sposób zmianę tła będziemy mieć połączoną z właściwością `isSelected` elementu. 

Żeby z niego skorzystać potrzebujemy jeszcze jednego elementu - `SelectionTracker`. W klasie `NumberListAdapter` dodajemy odpowiednie pole z `setterem`

```java
    private SelectionTracker<Long> selectionTracker;
    public void setSelectionTracker(SelectionTracker<Long> selectionTracker) {
        this.selectionTracker = selectionTracker;
    }
```

`SelectionTracker` umożliwia śledzenie wszystkich elementów zaznaczonych przez użytkownika i pozwala sprawdzić czy dany element jest zaznaczony czy nie. Zmodyfikujmy metodę `bind` klasy `ViewHolder` aby umożliwić śledzenie - robimy to poprzez ustawienie pola `isActivated` na `itemView`

```java
        public void bind(int value, boolean isActivated){
            textView.setText(String.valueOf(value));
            itemView.setActivated(isActivated);
        }
```

Teraz musimy zmodyfikować wywołanie w metodzie `onBindViewHolder`

```java
    @Override
    public void onBindViewHolder(@NonNull NumberListAdapter.NumberListViewHolder holder, int position) {
        holder.bind(numberList.get(position), selectionTracker.isSelected((long)position));
    }
```

Na instancji `SelectionTracker` wywołujemy metodę `isSelected` sptawdzającą czy element o zadanym `id` jest zaznaczony przez użytkownika. Jako `id` przekazujemy `position` zrzutowany na `long`.

Ostatnim krokiem jest utworzenie `SelectionTracker` w klasie `MainActivity` i połączenie go z naszym `RecyclerView`. 

```java
recyclerView.setAdapter(numberListAdapter);
SelectionTracker<Long> selectionTracker = new SelectionTracker.Builder<>(
    "numberSelection",
    recyclerView,
    new StableIdKeyProvider(recyclerView),
    new NumberItemDetailsLookup(recyclerView),
    StorageStrategy.createLongStorage()
).withSelectionPredicate(
    SelectionPredicates.createSelectAnything()
).build();
numberListAdapter.setSelectionTracker(selectionTracker);
```

Tutaj istotna jest kolejność wykonania
- tworzymy `RecyclerView`
- powiązujemy `Adapter` z `RecyclerView`
- powiązujemy `SelectionTracker` z `RecyclerView`

Aby utworzyć `SelectionTracker` wykorzystujemy `Builder`

```java
SelectionTracker<Long> selectionTracker = new SelectionTracker.Builder<>()
```

Przyjmuje on szereg argumentów
- `selectionId` - `String` jednoznacznie identyfikujący `SelectionTracker` dla danej aktywności
- `recyclerView` - instancja `RecyclerView` na którym wywołujemy `SelectionTracker`
- `keyProvider` - źródło kluczy po których rozróżniamy elementy - tutaj wykorzystaliśmy domyślną implementację klasy `StableIdKeyProvider`
- `detailsLookup` - źródło informacji o elementach `Recyclerview` (pozycja elementu, unikalny identyfikator elementu)
- `storage` - strategia przechowywania stanu - możemy przechować informację o tym, które elementy są aktualnie zaznaczone

Następnie wywołujemy funkcję `withSelectionPredicate`, która pozwala nam określić sposób w jaki stosujemy zaznaczenie (możemy określić ograniczenia). Tutaj wywołana jest metoda `createSelectionAnything` nie posiadająca żadnych ograniczeń - jednocześnie zezwala na zaznaczenie wielu elementów.

Na tym etapie możemy przetestować aplikację.

<img src="https://media1.giphy.com/media/7VOdnpyhosbBDIQPR0/giphy.gif?cid=790b76117ce18b657fd9003f6e29b87c453b5bfad7667dad&rid=giphy.gif&ct=g" width="200" />